In [87]:
import requests
import pandas as pd
import pymysql
import os
import datetime
from datetime import datetime

In [88]:
def save_file(file_name,data):
    with open(os.path.join(os.getcwd(),'files',file_name),'w') as file:
       file.write(data)
       

In [89]:
def request_api(url, params=None, key=None):

   if params is None:
      response = requests.get(url)
      data = response.json()

   elif params is not None:
      response = requests.get(url, params= params)
      data = response.json()[key]

   df = pd.DataFrame(data) 
   return df


In [93]:
# Url coin list
coins_url = "https://api.coingecko.com/api/v3/coins/list"

df_coins_list=request_api(coins_url, params=None)
bitcoin_id = df_coins_list.loc[df_coins_list["id"] == "bitcoin", "id"].iloc[0]

# Url historical market data 
url_currency_historical_market_data = "https://api.coingecko.com/api/v3/coins/{}/market_chart/range"
initial_date = int(datetime.strptime("2022-01-01", "%Y-%m-%d").timestamp())
final_date = int(datetime.strptime("2022-03-31", "%Y-%m-%d").timestamp())
params = {
    "vs_currency": "usd",
    "from": initial_date,
    "to": final_date,
}

df_bitcoin_history_market = request_api(url_currency_historical_market_data.format(bitcoin_id), params = params, key="prices")





In [94]:
#modificar nombre columnas
df_bitcoin_history_market.rename(columns= {0:'date',1:'usd price'}, inplace = True)
#moficicar la columna de de fecha
df_bitcoin_history_market['date']=df_bitcoin_history_market['date'].astype('datetime64[ns]')


In [95]:
df_bitcoin_history_market['date']=df_bitcoin_history_market['date'].astype('datetime64[ns]')

In [97]:
df_bitcoin_history_market['usd price']=df_bitcoin_history_market['usd price'].astype(float)

In [98]:
df_bitcoin_history_market


,date,usd price
0,1970-01-01 00:27:21.006362956,46880.270561
1,1970-01-01 00:27:21.010074279,46907.544324
2,1970-01-01 00:27:21.013512694,46874.726169
3,1970-01-01 00:27:21.016948259,47372.201889
4,1970-01-01 00:27:21.020554453,47128.848463
...,...,...
2132,1970-01-01 00:27:28.677737696,46964.659709
2133,1970-01-01 00:27:28.681253669,46972.965974
2134,1970-01-01 00:27:28.684949470,47063.365850
2135,1970-01-01 00:27:28.688574445,47222.112848


In [ ]:
# Obtener el precio de Bitcoin en USD y por fecha del primer trimestre de 2022
url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range"
params = {
    "vs_currency": "usd",
    "from": 1640995200,  # 1 de enero de 2022 (timestamp)
    "to": 1672444799,    # 31 de marzo de 2022 (timestamp)
}
response = requests.get(url, params=params)

# Convertir los datos a un DataFrame de Pandas
data = response.json()
df = pd.DataFrame(data["prices"], columns=["timestamp", "price"])
df["date"] = pd.to_datetime(df["timestamp"], unit='ms')
df = df[["date", "price"]]


# cursor = connection.cursor()
# cursor.execute("CREATE TABLE IF NOT EXISTS bitcoin_prices (date DATE, price FLOAT)")
# for index, row in df.iterrows():
#     cursor.execute("INSERT INTO bitcoin_prices (date, price) VALUES (%s, %s)", (row["date"], row["price"]))
# connection.commit()
# cursor.close()
# connection.close()


In [100]:
#Database Credentials
#Credentials
rds_host="127.0.0.1"
user="root"
password="t3st2023#"


In [101]:
# Guardar la información en la base de datos MySQL
conn = pymysql.connect(host=rds_host,
                        user=user,
                        passwd=password,
                        connect_timeout=30)


In [102]:
cursor = conn.cursor()


In [103]:
query_create_database="""CREATE DATABASE IF NOT EXISTS currencies;"""        
cursor.execute(query_create_database)
conn.commit()





In [104]:
#Create table
query_create_table = f"""CREATE TABLE IF NOT EXISTS currencies.bitcoin_market_history_price(
                                id INTEGER PRIMARY KEY AUTO_INCREMENT,
                                date DATETIME NOT NULL, 
                                `usd price` DECIMAL(10, 2) NOT NULL
                                );"""
cursor.execute(query_create_database)
conn.commit()

In [110]:
data_bitcoin_history_market= list(zip(df_bitcoin_history_market['date'], df_bitcoin_history_market['usd price']))
query_insert_history_market= """INSERT INTO currencies.bitcoin_market_history_price (date, `usd price`) VALUES (%s, %s);"""
cursor.executemany(query_insert_history_market, data_bitcoin_history_market)
conn.commit()

ProgrammingError: (1146, "Table 'currencies.bitcoin_market_history_price' doesn't exist")